In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
from d2lvit import *
import copy
from collections import OrderedDict

In [2]:
model = torch.load(os.path.join('..', 'models', 'untrained_torchvit', 'digit-model.pt'))

In [3]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

0.949


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 45))]))
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 0.5526, -0.6681, -0.7577,  0.2164,  0.3682, -1.1499,  0.3463, -0.7079,
         -0.1652,  0.2435, -0.3830, -0.1657, -1.4728, -1.3349, -0.5495, -0.2955,
          0.6322,  0.1923,  0.7365, -0.4278,  0.1388, -0.7619,  0.4257,  0.7028,
          0.1806, -0.0485, -0.2082, -1.1656, -0.1419, -1.0007, -0.2682,  0.1478,
         -1.2013,  0.6380,  0.2971,  0.9242, -0.7511, -0.0979, -0.4066, -0.1344,
          0.6459,  0.0149, -0.4649,  0.3968,  1.0637]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [9]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'untrained_torchvit_noprims'), digits_model, False, False)

Epoch 10 done, train loss: 0.0004 val acc: 0.9500
Epoch 20 done, train loss: 0.0001 val acc: 0.9520
Epoch 30 done, train loss: 0.0000 val acc: 0.9510
Epoch 40 done, train loss: 0.0000 val acc: 0.9495
Epoch 50 done, train loss: 0.0000 val acc: 0.9490
Epoch 60 done, train loss: 0.0000 val acc: 0.9490
Epoch 70 done, train loss: 0.0000 val acc: 0.9485
Epoch 80 done, train loss: 0.0000 val acc: 0.9475
Epoch 90 done, train loss: 0.0000 val acc: 0.9460
Epoch 100 done, train loss: 0.0000 val acc: 0.9460


In [10]:
model = torch.load(os.path.join('..', 'models', 'untrained_torchvit_noprims', 'tree-model.pt'))

In [11]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, digits_model, False, False)
print(train_acc)

1.0


In [12]:
val_acc = m.predict(model, val_loader, device, config, digits_model, False, False)
print(val_acc)

0.952
